In [1]:
import pickle
import os
import re
import numpy as np
from gensim.models import word2vec
import logging
import pandas as pd

In [2]:
import sys
sys.path.insert(0, '../Core-scripts/')

from parse_and_prepare import ProteinProteinInteractionClassifier as ppi
import file_readers as fr
import prediction as pred

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

In [4]:
random_seeds = [144, 235, 905, 2895, 3462, 4225, 5056, 5192, 7751, 7813]

In [5]:
# Yeast using old model (w/o skipgramms and hierarchical softmax)

yeast_strict = pickle.load(open('../../Results/Yeast/yeast_mentions_strict_real.pkl', 'rb'))
yeast_gen = pickle.load(open('../../Results/Yeast/yeast_mentions_gen_real.pkl', 'rb'))
yeast_be = pickle.load(open('../../Results/Yeast/yeast_mentions_be_real.pkl', 'rb'))

In [6]:
yeast_strict_model = pred.make_w2v_model(yeast_strict, 'yeast_strict')
yeast_gen_model = pred.make_w2v_model(yeast_gen, 'yeast_gen')
yeast_be_model = pred.make_w2v_model(yeast_be, 'yeast_be')

2017-06-20 10:08:54,522 : INFO : collecting all words and their counts
2017-06-20 10:08:54,523 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-20 10:08:54,556 : INFO : collected 11431 word types from a corpus of 188505 raw words and 7145 sentences
2017-06-20 10:08:54,557 : INFO : Loading a fresh vocabulary
2017-06-20 10:08:54,568 : INFO : min_count=7 retains 2632 unique words (23% of original 11431, drops 8799)
2017-06-20 10:08:54,569 : INFO : min_count=7 leaves 171509 word corpus (90% of original 188505, drops 16996)
2017-06-20 10:08:54,575 : INFO : deleting the raw counts dictionary of 11431 items
2017-06-20 10:08:54,577 : INFO : sample=0.001 downsamples 42 most-common words
2017-06-20 10:08:54,577 : INFO : downsampling leaves estimated 121427 word corpus (70.8% of prior 171509)
2017-06-20 10:08:54,578 : INFO : estimated required memory for 2632 words and 800 dimensions: 18160800 bytes
2017-06-20 10:08:54,587 : INFO : resetting layer weights
2017-06

Parsing datasets sentences
Training Word2Vec Model


2017-06-20 10:08:55,646 : INFO : PROGRESS: at 87.87% examples, 532566 words/s, in_qsize 11, out_qsize 1
2017-06-20 10:08:55,706 : INFO : worker thread finished; awaiting finish of 7 more threads
2017-06-20 10:08:55,720 : INFO : worker thread finished; awaiting finish of 6 more threads
2017-06-20 10:08:55,734 : INFO : worker thread finished; awaiting finish of 5 more threads
2017-06-20 10:08:55,736 : INFO : worker thread finished; awaiting finish of 4 more threads
2017-06-20 10:08:55,755 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-06-20 10:08:55,763 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-06-20 10:08:55,765 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-06-20 10:08:55,765 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-06-20 10:08:55,766 : INFO : training on 942525 raw words (606910 effective words) took 1.1s, 540880 effective words/s
2017-06-20 10:08:55,767 : INFO : precomputing

Parsing datasets sentences


2017-06-20 10:08:56,807 : INFO : collecting all words and their counts
2017-06-20 10:08:56,809 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-20 10:08:56,856 : INFO : PROGRESS: at sentence #10000, processed 241613 words, keeping 11132 word types
2017-06-20 10:08:56,915 : INFO : PROGRESS: at sentence #20000, processed 485125 words, keeping 14997 word types
2017-06-20 10:08:56,963 : INFO : PROGRESS: at sentence #30000, processed 726910 words, keeping 17660 word types
2017-06-20 10:08:57,005 : INFO : PROGRESS: at sentence #40000, processed 968796 words, keeping 19737 word types


...

2017-06-20 10:08:57,035 : INFO : collected 21147 word types from a corpus of 1155938 raw words and 47662 sentences
2017-06-20 10:08:57,036 : INFO : Loading a fresh vocabulary
2017-06-20 10:08:57,055 : INFO : min_count=7 retains 6774 unique words (32% of original 21147, drops 14373)
2017-06-20 10:08:57,056 : INFO : min_count=7 leaves 1127019 word corpus (97% of original 1155938, drops 28919)
2017-06-20 10:08:57,072 : INFO : deleting the raw counts dictionary of 21147 items
2017-06-20 10:08:57,074 : INFO : sample=0.001 downsamples 48 most-common words
2017-06-20 10:08:57,074 : INFO : downsampling leaves estimated 821525 word corpus (72.9% of prior 1127019)
2017-06-20 10:08:57,075 : INFO : estimated required memory for 6774 words and 800 dimensions: 46740600 bytes
2017-06-20 10:08:57,099 : INFO : resetting layer weights
2017-06-20 10:08:57,215 : INFO : training model with 8 workers on 6774 vocabulary and 800 features, using sg=0 hs=0 sample=0.001 negative=5 window=9
2017-06-20 10:08:57,21

Parsing datasets sentences


2017-06-20 10:09:10,408 : INFO : collecting all words and their counts
2017-06-20 10:09:10,409 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-20 10:09:10,449 : INFO : PROGRESS: at sentence #10000, processed 228848 words, keeping 12367 word types
2017-06-20 10:09:10,488 : INFO : PROGRESS: at sentence #20000, processed 458192 words, keeping 16853 word types
2017-06-20 10:09:10,527 : INFO : PROGRESS: at sentence #30000, processed 688735 words, keeping 20141 word types
2017-06-20 10:09:10,568 : INFO : PROGRESS: at sentence #40000, processed 918442 words, keeping 22989 word types


Training Word2Vec Model


2017-06-20 10:09:10,608 : INFO : PROGRESS: at sentence #50000, processed 1147651 words, keeping 25312 word types
2017-06-20 10:09:10,652 : INFO : PROGRESS: at sentence #60000, processed 1374968 words, keeping 27377 word types
2017-06-20 10:09:10,690 : INFO : PROGRESS: at sentence #70000, processed 1603857 words, keeping 29219 word types
2017-06-20 10:09:10,733 : INFO : PROGRESS: at sentence #80000, processed 1833366 words, keeping 30891 word types
2017-06-20 10:09:10,771 : INFO : PROGRESS: at sentence #90000, processed 2062791 words, keeping 32380 word types
2017-06-20 10:09:10,811 : INFO : PROGRESS: at sentence #100000, processed 2292331 words, keeping 33781 word types
2017-06-20 10:09:10,852 : INFO : PROGRESS: at sentence #110000, processed 2521512 words, keeping 35070 word types
2017-06-20 10:09:10,890 : INFO : PROGRESS: at sentence #120000, processed 2749293 words, keeping 36325 word types
2017-06-20 10:09:10,933 : INFO : PROGRESS: at sentence #130000, processed 2976619 words, keep

In [7]:
from sklearn.ensemble import RandomForestClassifier
def RF_classifier_bow(train_vector, test_vector, labels_train, labels_test, name, n_trees=100, ran_s=23):
    """Adding Later"""
    print ("Training the random forest...")

    # Initialize a Random Forest classifier with n_trees trees
    forest = RandomForestClassifier(n_estimators=int(n_trees), random_state=int(ran_s))

    # Fit the forest to the training set, using the bag of words as
    # features and the sentiment labels as the response variable
    #
    # This may take a few minutes to run
    forest = forest.fit(train_vector, labels_train)
    probs = forest.predict_proba(test_vector)[:, 1] #For ROC Curve

    # Use the random forest to make sentiment label predictions
    print ("Predicting based on model...")
    result = forest.predict(test_vector)
    
    #Metrics
    accuracy = metrics.accuracy_score(labels_test, result)
    roc_auc = metrics.roc_auc_score(labels_test, probs)
    class_report = metrics.classification_report(labels_test, result)
    
    #Print Model report:
    print(name, '\nModel Report')
    print(name, 'Accuracy: %.4g' % accuracy)
    print(name, 'AUC Score (Train): %f' % roc_auc)
    print(name, 'Report \n', class_report)
    
    return accuracy, roc_auc, result

In [9]:
def XGB_modelfit(alg, train_vecs, train_labels, test_vecs, test_labels, w2v_model_type, useTrainCV=True, cv_folds=5, feature_selection=False):

    if feature_selection:
        clf = ExtraTreesClassifier(n_estimators=400)
        clf = clf.fit(train_vecs, train_labels)
        model = SelectFromModel(clf, prefit=True)
        train_vecs = model.transform(train_vecs)
        test_vecs = model.transform(test_vecs)

#     if useTrainCV:
#         xgb_param = alg.get_xgb_params()
#         xgtrain = xgb.DMatrix(train_vecs,
#                               label=train_labels)
#         cvresult = xgb.cv(xgb_param,
#                           xgtrain,
#                           num_boost_round=alg.get_params()['n_estimators'],
#                           nfold=cv_folds,
#                           metrics='auc',
#                           early_stopping_rounds=50)
#         alg.set_params(n_estimators=cvresult.shape[0])

    #fit the algorithm on the data
    alg.fit(train_vecs, train_labels, eval_metric='auc')

    #Predict training set:
    test_preds = alg.predict(test_vecs)
    test_predictions = [round(val) for val in test_preds]
    test_predprob = alg.predict_proba(test_vecs)[:, 1]

    #Metrics
    accuracy = metrics.accuracy_score(test_labels, test_predictions)
    roc_auc = metrics.roc_auc_score(test_labels, test_predprob)
    class_report = metrics.classification_report(test_labels, test_predictions)

    #Print Model report:
    print(w2v_model_type, '\nModel Report')
    print(w2v_model_type, 'Accuracy: %.4g' % accuracy)
    print(w2v_model_type, 'AUC Score (Train): %f' % roc_auc)
    print(w2v_model_type, 'Report \n', class_report)
    
    return accuracy, roc_auc, test_predictions

In [31]:
import time
import numpy as np
name_of_result = 'yeast_W2V'
strict_data = yeast_strict
w2v_strict = yeast_strict_model
w2v_gen = yeast_gen_model
w2v_be = yeast_be_model
xgb_clf = XGBClassifier(learning_rate = 0.05,
                        n_estimators = 1000,
                        subsample=0.7,
                        colsample_bytree=0.8,
                        gamma=0,
                        min_child_weight=3,
                        max_depth=2,
                        seed=24)
start = time.time()

rf_accuracy = []
rf_auc_score = []
rf_pred_labels = []
xgb_accuracy = []
xgb_auc_score = []
xgb_pred_labels = []
true_test_labels = []
    
for seed in random_seeds:
    strict_list_SR = pred.make_models(strict_data,
                                      name_of_result+'_SR_'+str(seed),
                                      prev_model=w2v_strict,
                                      ran_state=seed,
                                      BOW=True)
    
    strict_final_list = [strict_list_SR]
    
    print ('\nPredicting\n')

    for entry in strict_final_list:
        xgb_accu, xgb_auc, xgb_labels = XGB_modelfit(xgb_clf, entry[0], entry[4], entry[1], entry[5], 'BOW XGB '+str(seed))
        rf_accu, rf_auc, rf_labels = RF_classifier_bow(entry[0], entry[1], entry[4], entry[5], 'BOW RF '+str(seed))
        xgb_accuracy.append(xgb_accu)
        xgb_auc_score.append(xgb_auc)
        xgb_pred_labels.extend(xgb_labels)
        rf_accuracy.append(rf_accu)
        rf_auc_score.append(rf_auc)
        rf_pred_labels.extend(rf_labels)
        true_test_labels += entry[5]
        
rf_avg_recall = metrics.recall_score(true_test_labels, rf_pred_labels, average='micro')
rf_avg_precision = metrics.precision_score(true_test_labels, rf_pred_labels, average='micro')
xgb_avg_recall = metrics.recall_score(true_test_labels, xgb_pred_labels, average='micro')
xgb_avg_precision = metrics.precision_score(true_test_labels, xgb_pred_labels, average='micro')

print('Mean RF accuracy - ', np.mean(rf_accuracy))
print('Mean RF AUC - ', np.mean(rf_auc_score))
print('Mean RF Recall - ', rf_avg_recall)
print('Mean RF Precision - ', rf_avg_precision)
print('Mean XGB accuracy - ', np.mean(xgb_accuracy))
print('Mean XGB AUC - ', np.mean(xgb_auc_score))
print('Mean XGB Recall - ', xgb_avg_recall)
print('Mean XGB Precision - ', xgb_avg_precision)

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 144 
Model Report
BOW XGB 144 Accuracy: 0.5
BOW XGB 144 AUC Score (Train): 0.604947
BOW XGB 144 Report 
              precision    recall  f1-score   support

          0       0.40      0.76      0.53        41
          1       0.71      0.35      0.47        71

avg / total       0.60      0.50      0.49       112

Training the random forest...
Predicting based on model...
BOW RF 144 
Model Report
BOW RF 144 Accuracy: 0.5268
BOW RF 144 AUC Score (Train): 0.571625
BOW RF 144 Report 
              precision    recall  f1-score   support

          0       0.42      0.73      0.53        41
          1       0.72      0.41      0.52        71

avg / total       0.61      0.53      0.53       112

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 235 
Model Report
BOW XGB 235 Accuracy: 0.6739
BOW XGB 235 AUC Score (Train): 0.754386
BOW XGB 235 Report 
              precision    recall  f1-score   support

          0       0.57      0.84      0.68        19
          1       0.83      0.56      0.67        27

avg / total       0.73      0.67      0.67        46

Training the random forest...
Predicting based on model...
BOW RF 235 
Model Report
BOW RF 235 Accuracy: 0.6957
BOW RF 235 AUC Score (Train): 0.742690
BOW RF 235 Report 
              precision    recall  f1-score   support

          0       0.61      0.74      0.67        19
          1       0.78      0.67      0.72        27

avg / total       0.71      0.70      0.70        46

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



...


Predicting

BOW XGB 5192 
Model Report
BOW XGB 5192 Accuracy: 0.6538
BOW XGB 5192 AUC Score (Train): 0.676974
BOW XGB 5192 Report 
              precision    recall  f1-score   support

          0       0.63      0.80      0.70        40
          1       0.70      0.50      0.58        38

avg / total       0.66      0.65      0.65        78

Training the random forest...
Predicting based on model...
BOW RF 5192 
Model Report
BOW RF 5192 Accuracy: 0.641
BOW RF 5192 AUC Score (Train): 0.607895
BOW RF 5192 Report 
              precision    recall  f1-score   support

          0       0.64      0.70      0.67        40
          1       0.65      0.58      0.61        38

avg / total       0.64      0.64      0.64        78

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 7751 
Model Report
BOW XGB 7751 Accuracy: 0.6176
BOW XGB 7751 AUC Score (Train): 0.701741
BOW XGB 7751 Report 
              precision    recall  f1-score   support

          0       0.61      0.82      0.70        55
          1       0.64      0.38      0.48        47

avg / total       0.62      0.62      0.60       102

Training the random forest...
Predicting based on model...
BOW RF 7751 
Model Report
BOW RF 7751 Accuracy: 0.6863
BOW RF 7751 AUC Score (Train): 0.720696
BOW RF 7751 Report 
              precision    recall  f1-score   support

          0       0.66      0.85      0.75        55
          1       0.74      0.49      0.59        47

avg / total       0.70      0.69      0.67       102

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 7813 
Model Report
BOW XGB 7813 Accuracy: 0.7288
BOW XGB 7813 AUC Score (Train): 0.715476
BOW XGB 7813 Report 
              precision    recall  f1-score   support

          0       0.73      0.86      0.79        35
          1       0.72      0.54      0.62        24

avg / total       0.73      0.73      0.72        59

Training the random forest...
Predicting based on model...
BOW RF 7813 
Model Report
BOW RF 7813 Accuracy: 0.661
BOW RF 7813 AUC Score (Train): 0.615476
BOW RF 7813 Report 
              precision    recall  f1-score   support

          0       0.67      0.83      0.74        35
          1       0.62      0.42      0.50        24

avg / total       0.65      0.66      0.64        59

Mean RF accuracy -  0.655367668226
Mean RF AUC -  0.692498420192
Mean RF Recall -  0.649606299213
Mean RF Precision -  0.649606299213
Mean XGB accuracy -  0.665629487916
Mean XGB AUC -  0.720845691995
Mean XGB Recall -  0.657480314961
Mean XGB Precision -  0.6574

In [32]:
import time
import numpy as np
name_of_result = 'yeast_W2V'
strict_data = yeast_strict
w2v_strict = yeast_strict_model
xgb_clf = XGBClassifier(learning_rate = 0.05,
                        n_estimators = 1000,
                        subsample=0.7,
                        colsample_bytree=0.8,
                        gamma=0,
                        min_child_weight=3,
                        max_depth=2,
                        seed=24)
start = time.time()

bow_accuracy = []
bow_auc_score = []
bow_pred_labels = []
w2v_accuracy_strict = []
w2v_auc_score_strict = []
w2v_pred_labels_strict = []
true_test_labels = []

for seed in random_seeds:
    strict_list_SR = pred.make_models(strict_data,
                                      name_of_result+'_SR_'+str(seed),
                                      prev_model=w2v_strict,
                                      ran_state=seed,
                                      BOW=True)

    strict_final_list = [strict_list_SR]
    
    print ('\nPredicting\n')

    for entry in strict_final_list:
        bow_accu, bow_auc, bow_labels = XGB_modelfit(xgb_clf, entry[0], entry[4], entry[1], entry[5], 'BOW XGB '+str(seed))
        w2v_accu, w2v_auc, w2v_labels  = XGB_modelfit(xgb_clf, entry[2], entry[4], entry[3], entry[5], 'W2V XGB strict '+str(seed))
        w2v_accuracy_strict.append(w2v_accu)
        w2v_auc_score_strict.append(w2v_auc)
        w2v_pred_labels_strict.extend(w2v_labels)
        bow_accuracy.append(bow_accu)
        bow_auc_score.append(bow_auc)
        bow_pred_labels.extend(bow_labels)
        true_test_labels += entry[5]
        
bow_avg_recall = metrics.recall_score(true_test_labels, bow_pred_labels, average='micro')
bow_avg_precision = metrics.precision_score(true_test_labels, bow_pred_labels, average='micro')
w2v_avg_recall_strict = metrics.recall_score(true_test_labels, w2v_pred_labels_strict, average='micro')
w2v_avg_precision_strict = metrics.precision_score(true_test_labels, w2v_pred_labels_strict, average='micro')

print('Mean BOW accuracy - ', np.mean(bow_accuracy))
print('Mean BOW AUC - ', np.mean(bow_auc_score))
print('Mean BOW Recall - ', bow_avg_recall)
print('Mean BOW Precision - ', bow_avg_precision)
print('Mean W2V accuracy - ', np.mean(w2v_accuracy_strict))
print('Mean W2V AUC - ', np.mean(w2v_auc_score_strict))
print('Mean W2V Recall - ', w2v_avg_recall_strict)
print('Mean W2V Precision - ', w2v_avg_precision_strict)

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 144 
Model Report
BOW XGB 144 Accuracy: 0.5
BOW XGB 144 AUC Score (Train): 0.604947
BOW XGB 144 Report 
              precision    recall  f1-score   support

          0       0.40      0.76      0.53        41
          1       0.71      0.35      0.47        71

avg / total       0.60      0.50      0.49       112

W2V XGB strict 144 
Model Report
W2V XGB strict 144 Accuracy: 0.4911
W2V XGB strict 144 AUC Score (Train): 0.541738
W2V XGB strict 144 Report 
              precision    recall  f1-score   support

          0       0.40      0.76      0.52        41
          1       0.71      0.34      0.46        71

avg / total       0.59      0.49      0.48       112

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 235 
Model Report
BOW XGB 235 Accuracy: 0.6739
BOW XGB 235 AUC Score (Train): 0.754386
BOW XGB 235 Report 
              precision    recall  f1-score   support

          0       0.57      0.84      0.68        19
          1       0.83      0.56      0.67        27

avg / total       0.73      0.67      0.67        46

W2V XGB strict 235 
Model Report
W2V XGB strict 235 Accuracy: 0.587
W2V XGB strict 235 AUC Score (Train): 0.692008
W2V XGB strict 235 Report 
              precision    recall  f1-score   support

          0       0.50      0.79      0.61        19
          1       0.75      0.44      0.56        27

avg / total       0.65      0.59      0.58        46

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



...


Predicting

BOW XGB 5192 
Model Report
BOW XGB 5192 Accuracy: 0.6538
BOW XGB 5192 AUC Score (Train): 0.676974
BOW XGB 5192 Report 
              precision    recall  f1-score   support

          0       0.63      0.80      0.70        40
          1       0.70      0.50      0.58        38

avg / total       0.66      0.65      0.65        78

W2V XGB strict 5192 
Model Report
W2V XGB strict 5192 Accuracy: 0.6282
W2V XGB strict 5192 AUC Score (Train): 0.658553
W2V XGB strict 5192 Report 
              precision    recall  f1-score   support

          0       0.62      0.70      0.66        40
          1       0.64      0.55      0.59        38

avg / total       0.63      0.63      0.63        78

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 7751 
Model Report
BOW XGB 7751 Accuracy: 0.6176
BOW XGB 7751 AUC Score (Train): 0.701741
BOW XGB 7751 Report 
              precision    recall  f1-score   support

          0       0.61      0.82      0.70        55
          1       0.64      0.38      0.48        47

avg / total       0.62      0.62      0.60       102

W2V XGB strict 7751 
Model Report
W2V XGB strict 7751 Accuracy: 0.7353
W2V XGB strict 7751 AUC Score (Train): 0.769826
W2V XGB strict 7751 Report 
              precision    recall  f1-score   support

          0       0.78      0.71      0.74        55
          1       0.69      0.77      0.73        47

avg / total       0.74      0.74      0.74       102

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 7813 
Model Report
BOW XGB 7813 Accuracy: 0.7288
BOW XGB 7813 AUC Score (Train): 0.715476
BOW XGB 7813 Report 
              precision    recall  f1-score   support

          0       0.73      0.86      0.79        35
          1       0.72      0.54      0.62        24

avg / total       0.73      0.73      0.72        59

W2V XGB strict 7813 
Model Report
W2V XGB strict 7813 Accuracy: 0.661
W2V XGB strict 7813 AUC Score (Train): 0.680952
W2V XGB strict 7813 Report 
              precision    recall  f1-score   support

          0       0.69      0.77      0.73        35
          1       0.60      0.50      0.55        24

avg / total       0.65      0.66      0.65        59

Mean BOW accuracy -  0.665629487916
Mean BOW AUC -  0.720845691995
Mean BOW Recall -  0.657480314961
Mean BOW Precision -  0.657480314961
Mean W2V accuracy -  0.6441985776
Mean W2V AUC -  0.681195462311
Mean W2V Recall -  0.646981627297
Mean W2V Precision -  0.646981627297


In [33]:
import time
import numpy as np
name_of_result = 'yeast_W2V'
strict_data = yeast_strict
w2v_gen = yeast_gen_model
xgb_clf = XGBClassifier(learning_rate = 0.05,
                        n_estimators = 1000,
                        subsample=0.7,
                        colsample_bytree=0.8,
                        gamma=0,
                        min_child_weight=3,
                        max_depth=2,
                        seed=24)
start = time.time()

bow_accuracy = []
bow_auc_score = []
bow_pred_labels = []
w2v_accuracy_gen = []
w2v_auc_score_gen = []
w2v_pred_labels_gen = []
true_test_labels = []

for seed in random_seeds:
    gen_list_SR = pred.make_models(strict_data,
                                      name_of_result+'_SR_'+str(seed),
                                      prev_model=w2v_gen,
                                      ran_state=seed,
                                      BOW=True)

    gen_final_list = [gen_list_SR]
    
    print ('\nPredicting\n')

    for entry in gen_final_list:
        bow_accu, bow_auc, bow_labels = XGB_modelfit(xgb_clf, entry[0], entry[4], entry[1], entry[5], 'BOW XGB '+str(seed))
        w2v_accu, w2v_auc, w2v_labels  = XGB_modelfit(xgb_clf, entry[2], entry[4], entry[3], entry[5], 'W2V XGB gen '+str(seed))
        w2v_accuracy_gen.append(w2v_accu)
        w2v_auc_score_gen.append(w2v_auc)
        w2v_pred_labels_gen.extend(w2v_labels)
        bow_accuracy.append(bow_accu)
        bow_auc_score.append(bow_auc)
        bow_pred_labels.extend(bow_labels)
        true_test_labels += entry[5]
        
bow_avg_recall = metrics.recall_score(true_test_labels, bow_pred_labels, average='micro')
bow_avg_precision = metrics.precision_score(true_test_labels, bow_pred_labels, average='micro')
w2v_avg_recall_gen = metrics.recall_score(true_test_labels, w2v_pred_labels_gen, average='micro')
w2v_avg_precision_gen = metrics.precision_score(true_test_labels, w2v_pred_labels_gen, average='micro')

print('Mean BOW accuracy - ', np.mean(bow_accuracy))
print('Mean BOW AUC - ', np.mean(bow_auc_score))
print('Mean BOW Recall - ', bow_avg_recall)
print('Mean BOW Precision - ', bow_avg_precision)
print('Mean W2V accuracy - ', np.mean(w2v_accuracy_gen))
print('Mean W2V AUC - ', np.mean(w2v_auc_score_gen))
print('Mean W2V Recall - ', w2v_avg_recall_gen)
print('Mean W2V Precision - ', w2v_avg_precision_gen)

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles


Predicting

BOW XGB 144 
Model Report
BOW XGB 144 Accuracy: 0.5
BOW XGB 144 AUC Score (Train): 0.604947
BOW XGB 144 Report 
              precision    recall  f1-score   support

          0       0.40      0.76      0.53        41
          1       0.71      0.35      0.47        71

avg / total       0.60      0.50      0.49       112

W2V XGB gen 144 
Model Report
W2V XGB gen 144 Accuracy: 0.5357
W2V XGB gen 144 AUC Score (Train): 0.607008
W2V XGB gen 144 Report 
              precision    recall  f1-score   support

          0       0.42      0.71      0.53        41
          1       0.72      0.44      0.54        71

avg / total       0.61      0.54      0.54       112

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles


Predicting

BOW XGB 235 
Model Report
BOW XGB 235 Accuracy: 0.6739
BOW XGB 235 AUC Score (Train): 0.754386
BOW XGB 235 Re

In [34]:
import time
import numpy as np
name_of_result = 'yeast_W2V'
strict_data = yeast_strict
w2v_be = yeast_be_model
xgb_clf = XGBClassifier(subsample=0.7,
                        colsample_bytree=0.8,
                        gamma=0,
                        min_child_weight=3,
                        max_depth=2,
                        seed=24)
start = time.time()

bow_accuracy = []
bow_auc_score = []
bow_pred_labels = []
w2v_accuracy_be = []
w2v_auc_score_be = []
w2v_pred_labels_be = []
true_test_labels = []

for seed in random_seeds:
    be_list_SR = pred.make_models(strict_data,
                                      name_of_result+'_SR_'+str(seed),
                                      prev_model=w2v_be,
                                      ran_state=seed,
                                      BOW=True)

    be_final_list = [be_list_SR]
    
    print ('\nPredicting\n')

    for entry in be_final_list:
        bow_accu, bow_auc, bow_labels = XGB_modelfit(xgb_clf, entry[0], entry[4], entry[1], entry[5], 'BOW XGB '+str(seed))
        w2v_accu, w2v_auc, w2v_labels  = XGB_modelfit(xgb_clf, entry[2], entry[4], entry[3], entry[5], 'W2V XGB be '+str(seed))
        w2v_accuracy_be.append(w2v_accu)
        w2v_auc_score_be.append(w2v_auc)
        w2v_pred_labels_be.extend(w2v_labels)
        bow_accuracy.append(bow_accu)
        bow_auc_score.append(bow_auc)
        bow_pred_labels.extend(bow_labels)
        true_test_labels += entry[5]
        
bow_avg_recall = metrics.recall_score(true_test_labels, bow_pred_labels, average='micro')
bow_avg_precision = metrics.precision_score(true_test_labels, bow_pred_labels, average='micro')
w2v_avg_recall_be = metrics.recall_score(true_test_labels, w2v_pred_labels_be, average='micro')
w2v_avg_precision_be = metrics.precision_score(true_test_labels, w2v_pred_labels_be, average='micro')

print('Mean BOW accuracy - ', np.mean(bow_accuracy))
print('Mean BOW AUC - ', np.mean(bow_auc_score))
print('Mean BOW Recall - ', bow_avg_recall)
print('Mean BOW Precision - ', bow_avg_precision)
print('Mean W2V accuracy - ', np.mean(w2v_accuracy_be))
print('Mean W2V AUC - ', np.mean(w2v_auc_score_be))
print('Mean W2V Recall - ', w2v_avg_recall_be)
print('Mean W2V Precision - ', w2v_avg_precision_be)

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles


Predicting

BOW XGB 144 
Model Report
BOW XGB 144 Accuracy: 0.5
BOW XGB 144 AUC Score (Train): 0.604947
BOW XGB 144 Report 
              precision    recall  f1-score   support

          0       0.40      0.76      0.53        41
          1       0.71      0.35      0.47        71

avg / total       0.60      0.50      0.49       112

W2V XGB be 144 
Model Report
W2V XGB be 144 Accuracy: 0.5714
W2V XGB be 144 AUC Score (Train): 0.644109
W2V XGB be 144 Report 
              precision    recall  f1-score   support

          0       0.45      0.78      0.57        41
          1       0.78      0.45      0.57        71

avg / total       0.66      0.57      0.57       112

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles


Predicting

BOW XGB 235 
Model Report
BOW XGB 235 Accuracy: 0.6739
BOW XGB 235 AUC Score (Train): 0.754386
BOW XGB 235 Report

In [30]:
a = [rf_accuracy, rf_auc_score, rf_pred_labels, 
     bow_accuracy, bow_auc_score, bow_pred_labels, 
     w2v_accuracy_strict, w2v_auc_score_strict, w2v_pred_labels_strict,
     w2v_accuracy_gen, w2v_auc_score_gen, w2v_pred_labels_gen,
     w2v_accuracy_be, w2v_auc_score_be, w2v_pred_labels_be,
     true_test_labels]
pickle.dump(a, open('BOW_vs_W2V.pkl', 'wb'))

In [20]:
import time
import numpy as np
name_of_result = 'yeast_W2V'
strict_data = yeast_strict
w2v_strict = yeast_strict_model
w2v_gen = yeast_gen_model
w2v_be = yeast_be_model
xgb_clf = XGBClassifier(seed=24)
start = time.time()

rf_accuracy = []
rf_auc_score = []
rf_pred_labels = []
xgb_accuracy = []
xgb_auc_score = []
xgb_pred_labels = []
true_test_labels = []
    
for seed in random_seeds:
    strict_list_SR = pred.make_models(strict_data,
                                      name_of_result+'_SR_'+str(seed),
                                      prev_model=w2v_strict,
                                      ran_state=seed,
                                      BOW=True)
    
    strict_final_list = [strict_list_SR]
    
    print ('\nPredicting\n')

    for entry in strict_final_list:
        xgb_accu, xgb_auc, xgb_labels = XGB_modelfit(xgb_clf, entry[0], entry[4], entry[1], entry[5], 'BOW XGB '+str(seed))
        rf_accu, rf_auc, rf_labels = RF_classifier_bow(entry[0], entry[1], entry[4], entry[5], 'BOW RF '+str(seed))
        xgb_accuracy.append(xgb_accu)
        xgb_auc_score.append(xgb_auc)
        xgb_pred_labels.extend(xgb_labels)
        rf_accuracy.append(rf_accu)
        rf_auc_score.append(rf_auc)
        rf_pred_labels.extend(rf_labels)
        true_test_labels += entry[5]
        
rf_avg_recall = metrics.recall_score(true_test_labels, rf_pred_labels, average='micro')
rf_avg_precision = metrics.precision_score(true_test_labels, rf_pred_labels, average='micro')
xgb_avg_recall = metrics.recall_score(true_test_labels, xgb_pred_labels, average='micro')
xgb_avg_precision = metrics.precision_score(true_test_labels, xgb_pred_labels, average='micro')

print('Mean RF accuracy - ', np.mean(rf_accuracy))
print('Mean RF AUC - ', np.mean(rf_auc_score))
print('Mean RF Recall - ', rf_avg_recall)
print('Mean RF Precision - ', rf_avg_precision)
print('Mean XGB accuracy - ', np.mean(xgb_accuracy))
print('Mean XGB AUC - ', np.mean(xgb_auc_score))
print('Mean XGB Recall - ', xgb_avg_recall)
print('Mean XGB Precision - ', xgb_avg_precision)

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 144 
Model Report
BOW XGB 144 Accuracy: 0.4107
BOW XGB 144 AUC Score (Train): 0.592408
BOW XGB 144 Report 
              precision    recall  f1-score   support

          0       0.37      0.85      0.51        41
          1       0.65      0.15      0.25        71

avg / total       0.55      0.41      0.35       112

Training the random forest...
Predicting based on model...
BOW RF 144 
Model Report
BOW RF 144 Accuracy: 0.5268
BOW RF 144 AUC Score (Train): 0.571625
BOW RF 144 Report 
              precision    recall  f1-score   support

          0       0.42      0.73      0.53        41
          1       0.72      0.41      0.52        71

avg / total       0.61      0.53      0.53       112

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 235 
Model Report
BOW XGB 235 Accuracy: 0.7174
BOW XGB 235 AUC Score (Train): 0.786550
BOW XGB 235 Report 
              precision    recall  f1-score   support

          0       0.61      0.89      0.72        19
          1       0.89      0.59      0.71        27

avg / total       0.77      0.72      0.72        46

Training the random forest...
Predicting based on model...
BOW RF 235 
Model Report
BOW RF 235 Accuracy: 0.6957
BOW RF 235 AUC Score (Train): 0.742690
BOW RF 235 Report 
              precision    recall  f1-score   support

          0       0.61      0.74      0.67        19
          1       0.78      0.67      0.72        27

avg / total       0.71      0.70      0.70        46

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



...


Predicting

BOW XGB 5192 
Model Report
BOW XGB 5192 Accuracy: 0.5641
BOW XGB 5192 AUC Score (Train): 0.654934
BOW XGB 5192 Report 
              precision    recall  f1-score   support

          0       0.55      0.80      0.65        40
          1       0.60      0.32      0.41        38

avg / total       0.58      0.56      0.54        78

Training the random forest...
Predicting based on model...
BOW RF 5192 
Model Report
BOW RF 5192 Accuracy: 0.641
BOW RF 5192 AUC Score (Train): 0.607895
BOW RF 5192 Report 
              precision    recall  f1-score   support

          0       0.64      0.70      0.67        40
          1       0.65      0.58      0.61        38

avg / total       0.64      0.64      0.64        78

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 7751 
Model Report
BOW XGB 7751 Accuracy: 0.598
BOW XGB 7751 AUC Score (Train): 0.689942
BOW XGB 7751 Report 
              precision    recall  f1-score   support

          0       0.58      0.91      0.71        55
          1       0.69      0.23      0.35        47

avg / total       0.63      0.60      0.54       102

Training the random forest...
Predicting based on model...
BOW RF 7751 
Model Report
BOW RF 7751 Accuracy: 0.6863
BOW RF 7751 AUC Score (Train): 0.720696
BOW RF 7751 Report 
              precision    recall  f1-score   support

          0       0.66      0.85      0.75        55
          1       0.74      0.49      0.59        47

avg / total       0.70      0.69      0.67       102

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 7813 
Model Report
BOW XGB 7813 Accuracy: 0.6102
BOW XGB 7813 AUC Score (Train): 0.676190
BOW XGB 7813 Report 
              precision    recall  f1-score   support

          0       0.64      0.80      0.71        35
          1       0.53      0.33      0.41        24

avg / total       0.59      0.61      0.59        59

Training the random forest...
Predicting based on model...
BOW RF 7813 
Model Report
BOW RF 7813 Accuracy: 0.661
BOW RF 7813 AUC Score (Train): 0.615476
BOW RF 7813 Report 
              precision    recall  f1-score   support

          0       0.67      0.83      0.74        35
          1       0.62      0.42      0.50        24

avg / total       0.65      0.66      0.64        59

Mean RF accuracy -  0.655367668226
Mean RF AUC -  0.692498420192
Mean RF Recall -  0.649606299213
Mean RF Precision -  0.649606299213
Mean XGB accuracy -  0.618467396696
Mean XGB AUC -  0.702194185072
Mean XGB Recall -  0.608923884514
Mean XGB Precision -  0.6089

In [21]:
import time
import numpy as np
name_of_result = 'yeast_W2V'
strict_data = yeast_strict
w2v_strict = yeast_strict_model
xgb_clf = XGBClassifier(seed=24)
start = time.time()

bow_accuracy = []
bow_auc_score = []
bow_pred_labels = []
w2v_accuracy_strict = []
w2v_auc_score_strict = []
w2v_pred_labels_strict = []
true_test_labels = []

for seed in random_seeds:
    strict_list_SR = pred.make_models(strict_data,
                                      name_of_result+'_SR_'+str(seed),
                                      prev_model=w2v_strict,
                                      ran_state=seed,
                                      BOW=True)

    strict_final_list = [strict_list_SR]
    
    print ('\nPredicting\n')

    for entry in strict_final_list:
        bow_accu, bow_auc, bow_labels = XGB_modelfit(xgb_clf, entry[0], entry[4], entry[1], entry[5], 'BOW XGB '+str(seed))
        w2v_accu, w2v_auc, w2v_labels  = XGB_modelfit(xgb_clf, entry[2], entry[4], entry[3], entry[5], 'W2V XGB strict '+str(seed))
        w2v_accuracy_strict.append(w2v_accu)
        w2v_auc_score_strict.append(w2v_auc)
        w2v_pred_labels_strict.extend(w2v_labels)
        bow_accuracy.append(bow_accu)
        bow_auc_score.append(bow_auc)
        bow_pred_labels.extend(bow_labels)
        true_test_labels += entry[5]
        
bow_avg_recall = metrics.recall_score(true_test_labels, bow_pred_labels, average='micro')
bow_avg_precision = metrics.precision_score(true_test_labels, bow_pred_labels, average='micro')
w2v_avg_recall_strict = metrics.recall_score(true_test_labels, w2v_pred_labels_strict, average='micro')
w2v_avg_precision_strict = metrics.precision_score(true_test_labels, w2v_pred_labels_strict, average='micro')

print('Mean BOW accuracy - ', np.mean(bow_accuracy))
print('Mean BOW AUC - ', np.mean(bow_auc_score))
print('Mean BOW Recall - ', bow_avg_recall)
print('Mean BOW Precision - ', bow_avg_precision)
print('Mean W2V accuracy - ', np.mean(w2v_accuracy_strict))
print('Mean W2V AUC - ', np.mean(w2v_auc_score_strict))
print('Mean W2V Recall - ', w2v_avg_recall_strict)
print('Mean W2V Precision - ', w2v_avg_precision_strict)

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 144 
Model Report
BOW XGB 144 Accuracy: 0.4107
BOW XGB 144 AUC Score (Train): 0.592408
BOW XGB 144 Report 
              precision    recall  f1-score   support

          0       0.37      0.85      0.51        41
          1       0.65      0.15      0.25        71

avg / total       0.55      0.41      0.35       112

W2V XGB strict 144 
Model Report
W2V XGB strict 144 Accuracy: 0.4643
W2V XGB strict 144 AUC Score (Train): 0.536585
W2V XGB strict 144 Report 
              precision    recall  f1-score   support

          0       0.38      0.76      0.51        41
          1       0.68      0.30      0.41        71

avg / total       0.57      0.46      0.45       112

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 235 
Model Report
BOW XGB 235 Accuracy: 0.7174
BOW XGB 235 AUC Score (Train): 0.786550
BOW XGB 235 Report 
              precision    recall  f1-score   support

          0       0.61      0.89      0.72        19
          1       0.89      0.59      0.71        27

avg / total       0.77      0.72      0.72        46

W2V XGB strict 235 
Model Report
W2V XGB strict 235 Accuracy: 0.6087
W2V XGB strict 235 AUC Score (Train): 0.719298
W2V XGB strict 235 Report 
              precision    recall  f1-score   support

          0       0.52      0.74      0.61        19
          1       0.74      0.52      0.61        27

avg / total       0.65      0.61      0.61        46

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



...


Predicting

BOW XGB 5192 
Model Report
BOW XGB 5192 Accuracy: 0.5641
BOW XGB 5192 AUC Score (Train): 0.654934
BOW XGB 5192 Report 
              precision    recall  f1-score   support

          0       0.55      0.80      0.65        40
          1       0.60      0.32      0.41        38

avg / total       0.58      0.56      0.54        78

W2V XGB strict 5192 
Model Report
W2V XGB strict 5192 Accuracy: 0.6282
W2V XGB strict 5192 AUC Score (Train): 0.669079
W2V XGB strict 5192 Report 
              precision    recall  f1-score   support

          0       0.61      0.75      0.67        40
          1       0.66      0.50      0.57        38

avg / total       0.63      0.63      0.62        78

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 7751 
Model Report
BOW XGB 7751 Accuracy: 0.598
BOW XGB 7751 AUC Score (Train): 0.689942
BOW XGB 7751 Report 
              precision    recall  f1-score   support

          0       0.58      0.91      0.71        55
          1       0.69      0.23      0.35        47

avg / total       0.63      0.60      0.54       102

W2V XGB strict 7751 
Model Report
W2V XGB strict 7751 Accuracy: 0.7353
W2V XGB strict 7751 AUC Score (Train): 0.793424
W2V XGB strict 7751 Report 
              precision    recall  f1-score   support

          0       0.75      0.76      0.76        55
          1       0.72      0.70      0.71        47

avg / total       0.73      0.74      0.74       102

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 7813 
Model Report
BOW XGB 7813 Accuracy: 0.6102
BOW XGB 7813 AUC Score (Train): 0.676190
BOW XGB 7813 Report 
              precision    recall  f1-score   support

          0       0.64      0.80      0.71        35
          1       0.53      0.33      0.41        24

avg / total       0.59      0.61      0.59        59

W2V XGB strict 7813 
Model Report
W2V XGB strict 7813 Accuracy: 0.678
W2V XGB strict 7813 AUC Score (Train): 0.665476
W2V XGB strict 7813 Report 
              precision    recall  f1-score   support

          0       0.71      0.77      0.74        35
          1       0.62      0.54      0.58        24

avg / total       0.67      0.68      0.67        59

Mean BOW accuracy -  0.618467396696
Mean BOW AUC -  0.702194185072
Mean BOW Recall -  0.608923884514
Mean BOW Precision -  0.608923884514
Mean W2V accuracy -  0.634782086507
Mean W2V AUC -  0.689453385783
Mean W2V Recall -  0.636482939633
Mean W2V Precision -  0.636482939633


In [22]:
import time
import numpy as np
name_of_result = 'yeast_W2V'
strict_data = yeast_strict
w2v_gen = yeast_gen_model
xgb_clf = XGBClassifier(seed=24)
start = time.time()

bow_accuracy = []
bow_auc_score = []
bow_pred_labels = []
w2v_accuracy_gen = []
w2v_auc_score_gen = []
w2v_pred_labels_gen = []
true_test_labels = []

for seed in random_seeds:
    gen_list_SR = pred.make_models(strict_data,
                                      name_of_result+'_SR_'+str(seed),
                                      prev_model=w2v_gen,
                                      ran_state=seed,
                                      BOW=True)

    gen_final_list = [gen_list_SR]
    
    print ('\nPredicting\n')

    for entry in gen_final_list:
        bow_accu, bow_auc, bow_labels = XGB_modelfit(xgb_clf, entry[0], entry[4], entry[1], entry[5], 'BOW XGB '+str(seed))
        w2v_accu, w2v_auc, w2v_labels  = XGB_modelfit(xgb_clf, entry[2], entry[4], entry[3], entry[5], 'W2V XGB gen '+str(seed))
        w2v_accuracy_gen.append(w2v_accu)
        w2v_auc_score_gen.append(w2v_auc)
        w2v_pred_labels_gen.extend(w2v_labels)
        bow_accuracy.append(bow_accu)
        bow_auc_score.append(bow_auc)
        bow_pred_labels.extend(bow_labels)
        true_test_labels += entry[5]
        
bow_avg_recall = metrics.recall_score(true_test_labels, bow_pred_labels, average='micro')
bow_avg_precision = metrics.precision_score(true_test_labels, bow_pred_labels, average='micro')
w2v_avg_recall_gen = metrics.recall_score(true_test_labels, w2v_pred_labels_gen, average='micro')
w2v_avg_precision_gen = metrics.precision_score(true_test_labels, w2v_pred_labels_gen, average='micro')

print('Mean BOW accuracy - ', np.mean(bow_accuracy))
print('Mean BOW AUC - ', np.mean(bow_auc_score))
print('Mean BOW Recall - ', bow_avg_recall)
print('Mean BOW Precision - ', bow_avg_precision)
print('Mean W2V accuracy - ', np.mean(w2v_accuracy_gen))
print('Mean W2V AUC - ', np.mean(w2v_auc_score_gen))
print('Mean W2V Recall - ', w2v_avg_recall_gen)
print('Mean W2V Precision - ', w2v_avg_precision_gen)

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles


Predicting

BOW XGB 144 
Model Report
BOW XGB 144 Accuracy: 0.4107
BOW XGB 144 AUC Score (Train): 0.592408
BOW XGB 144 Report 
              precision    recall  f1-score   support

          0       0.37      0.85      0.51        41
          1       0.65      0.15      0.25        71

avg / total       0.55      0.41      0.35       112

W2V XGB gen 144 
Model Report
W2V XGB gen 144 Accuracy: 0.5536
W2V XGB gen 144 AUC Score (Train): 0.602542
W2V XGB gen 144 Report 
              precision    recall  f1-score   support

          0       0.44      0.78      0.56        41
          1       0.77      0.42      0.55        71

avg / total       0.65      0.55      0.55       112

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles


Predicting

BOW XGB 235 
Model Report
BOW XGB 235 Accuracy: 0.7174
BOW XGB 235 AUC Score (Train): 0.786550
BOW XGB 235

In [23]:
import time
import numpy as np
name_of_result = 'yeast_W2V'
strict_data = yeast_strict
w2v_be = yeast_be_model
xgb_clf = XGBClassifier(seed=24)
start = time.time()

bow_accuracy = []
bow_auc_score = []
bow_pred_labels = []
w2v_accuracy_be = []
w2v_auc_score_be = []
w2v_pred_labels_be = []
true_test_labels = []

for seed in random_seeds:
    be_list_SR = pred.make_models(strict_data,
                                      name_of_result+'_SR_'+str(seed),
                                      prev_model=w2v_be,
                                      ran_state=seed,
                                      BOW=True)

    be_final_list = [be_list_SR]
    
    print ('\nPredicting\n')

    for entry in be_final_list:
        bow_accu, bow_auc, bow_labels = XGB_modelfit(xgb_clf, entry[0], entry[4], entry[1], entry[5], 'BOW XGB '+str(seed))
        w2v_accu, w2v_auc, w2v_labels  = XGB_modelfit(xgb_clf, entry[2], entry[4], entry[3], entry[5], 'W2V XGB be '+str(seed))
        w2v_accuracy_be.append(w2v_accu)
        w2v_auc_score_be.append(w2v_auc)
        w2v_pred_labels_be.extend(w2v_labels)
        bow_accuracy.append(bow_accu)
        bow_auc_score.append(bow_auc)
        bow_pred_labels.extend(bow_labels)
        true_test_labels += entry[5]
        
bow_avg_recall = metrics.recall_score(true_test_labels, bow_pred_labels, average='micro')
bow_avg_precision = metrics.precision_score(true_test_labels, bow_pred_labels, average='micro')
w2v_avg_recall_be = metrics.recall_score(true_test_labels, w2v_pred_labels_be, average='micro')
w2v_avg_precision_be = metrics.precision_score(true_test_labels, w2v_pred_labels_be, average='micro')

print('Mean BOW accuracy - ', np.mean(bow_accuracy))
print('Mean BOW AUC - ', np.mean(bow_auc_score))
print('Mean BOW Recall - ', bow_avg_recall)
print('Mean BOW Precision - ', bow_avg_precision)
print('Mean W2V accuracy - ', np.mean(w2v_accuracy_be))
print('Mean W2V AUC - ', np.mean(w2v_auc_score_be))
print('Mean W2V Recall - ', w2v_avg_recall_be)
print('Mean W2V Precision - ', w2v_avg_precision_be)

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles


Predicting

BOW XGB 144 
Model Report
BOW XGB 144 Accuracy: 0.4107
BOW XGB 144 AUC Score (Train): 0.592408
BOW XGB 144 Report 
              precision    recall  f1-score   support

          0       0.37      0.85      0.51        41
          1       0.65      0.15      0.25        71

avg / total       0.55      0.41      0.35       112

W2V XGB be 144 
Model Report
W2V XGB be 144 Accuracy: 0.5357
W2V XGB be 144 AUC Score (Train): 0.616627
W2V XGB be 144 Report 
              precision    recall  f1-score   support

          0       0.42      0.76      0.54        41
          1       0.74      0.41      0.53        71

avg / total       0.63      0.54      0.53       112

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles


Predicting

BOW XGB 235 
Model Report
BOW XGB 235 Accuracy: 0.7174
BOW XGB 235 AUC Score (Train): 0.786550
BOW XGB 235 Rep